Dataset: https://www.kaggle.com/huan9huan/walk-or-run

In [ ]:
import numpy as np 
import os
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch import Tensor

class Object(object): 
    pass

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

In [ ]:
orig = Object()

In [ ]:
orig.model = models.resnet50(pretrained=True).to(device)

In [ ]:
orig.model

In [ ]:
orig.model.fc = nn.Sequential()
_ = orig.model.requires_grad_(False)
_ = orig.model.eval()

## Prepare Images

Reorgnize images on the disk so that they can be quickly downloaded by ImageFolder/ImageLoader.

```
 /train/
    label1/
       train_1.jpg
       train_2.jpg
    label2/
       ...
 /test
    /label1/
       ...
    /label2/
```

In [ ]:
import os, csv, shutil

IMAGE_PATH="../input/plant-pathology-2020-fgvc7"

def move_files(root_path):
    image_class = {}
    classes = []
    class_pos = {}
    class_image = {}
    
    csv_name = 'train.csv'
    with open(f"{root_path}/{csv_name}", "rt") as f:
        first = True
        for line in csv.reader(f):
            if first:
                for i, c in enumerate(line):
                    if i > 0:
                        class_pos.setdefault(i - 1, c)
                        classes.append(c)
            else:
                for i, c in enumerate(line):
                    if i > 0 and int(c) == 1:
                        file_name = line[0]
                        image_class.setdefault(file_name, i - 1)
                        img_class_name = class_pos[i - 1]
                        class_image.setdefault(img_class_name, [])
                        class_image[img_class_name].append(file_name)
            first = False
                        
    types = ["train", "test"]
    for t in types:
        dir_path = f"{root_path}/{t}"
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        for c in classes:
            sub_path = dir_path + "/" + c
            if not os.path.exists(sub_path):
                os.makedirs(sub_path)

    # files into test: 70 or 20%, whicheve is smaller
    for class_name in class_image:
        images = class_image[class_name]
        test_size = int(min(70, 0.2 * len(images)))
        sub_path = f"{root_path}/train/{class_name}"
        for fname in images[test_size : ]:
            src = f"{root_path}/images/{fname}.jpg"
            dst = f"{sub_path}/{fname}.jpg"
            shutil.copyfile(src, dst)
        sub_path = f"{root_path}/test/{class_name}"
        for fname in images[: test_size]:
            src = f"{root_path}/images/{fname}.jpg"
            dst = f"{sub_path}/{fname}.jpg"
            shutil.copyfile(src, dst)

In [ ]:
move_files(IMAGE_PATH)

# Load Images

In [ ]:
input_path = "../input/walk-or-run"

In [ ]:
tr = Object()

In [ ]:
tr.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, .224, .225])
tr.transforms_train = transforms.Compose([
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    tr.normalize
])
tr.transforms_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    tr.normalize
])

In [ ]:
tr.train = datasets.ImageFolder(input_path + '/walk_or_run_train/train', transform=tr.transforms_train)
tr.valid = datasets.ImageFolder(input_path + '/walk_or_run_test/test', transform=tr.transforms_valid)

In [ ]:
len(tr.train.samples)

In [ ]:
tr.train_loader = torch.utils.data.DataLoader(tr.train, batch_size=32, num_workers=4)
tr.valid_loader = torch.utils.data.DataLoader(tr.valid, batch_size=32, num_workers=4)

In [ ]:
tr.train_loader.dataset.classes

Load images in batches, calculate, and store intermediate outputs for each image.

In [ ]:
%%time

tr.train_embeddings = []
tr.y = []

# The loop will read the set of images and corresponding
# labels one batch at a time, or 16 images/labels at a time.
# (see the batch_size value above for the actual size).
for x, y in tr.train_loader:
    # x.shape == [16, 3, 224, 224]
    # y.shape == [16]
    
    # Calculate outputs for all 16 images in a batch.
    # Rembember that outputs are the values produce by the
    # layer one before the last. Typically these values 
    # are called "embeddings".
    # It will be an array of 2048 real values per image.
    tr.batch_embeddings = orig.model(x.to(device))
    print(tr.batch_embeddings.shape, y)
    
    # Eventually we want to create a continious tensor of shape
    # [600, 2038] for all embeddings. To do this we need to break
    # down batches into the list of individual tensors, e.g.
    # [16, 2048] -> [[2048], [2048], ...]
    
    tr.batch_embeddings = tr.batch_embeddings.unbind()
    
    # Accumulate embeddings in a list
    tr.train_embeddings.extend(tr.batch_embeddings)
    tr.y.extend(y.unbind())
    
    print(len(tr.train_embeddings), len(tr.y))

In [ ]:
tr.train_embeddings[0]

In [ ]:
tr.train_embeddings = torch.stack(tr.train_embeddings)
tr.train_y = torch.stack(tr.y)
tr.train_embeddings.shape, tr.train_y.shape

Read the test set:

In [ ]:
%%time

tr.valid_embeddings = []
tr.valid_y = []
tr.X_valid = []

for x, y in tr.valid_loader:
    tr.batch_embeddings = orig.model(x.to(device)).unbind()
    tr.valid_embeddings.extend(tr.batch_embeddings)
    tr.valid_y.extend(y.unbind())
    tr.X_valid.extend(x.unbind())
    
tr.batch_embeddings = None
tr.valid_embeddings = torch.stack(tr.valid_embeddings)
tr.valid_y = torch.stack(tr.valid_y)
tr.X_valid = torch.stack(tr.X_valid)

print(tr.X_valid.shape, tr.valid_embeddings.shape, tr.valid_y.shape)

An embedding for each image is an array of 2048 floats:

In [ ]:
tr.valid_embeddings[0]

In [ ]:
def train_embeddings(model, criterion, opt, epochs, train_emb, valid_emd, train_y, valid_y):
    for epoch in range(epochs):
        print(f'Epoch {epoch+1} of {epochs}')
        print('******************************')
        model.train()
        running_loss = 0
        running_correct = 0
        
        x = train_emb
        y = train_y
        outputs = model(x)
        loss = criterion(outputs, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * x.size(0)
        running_correct += torch.sum(preds==y.data)
        epoch_loss = running_loss / train_emb.shape[0]
        epoch_acc = running_correct.double() / train_emb.shape[0]
        print('Training loss: {:.4f}, accuracy: {:.4f}'.format(epoch_loss, epoch_acc))
        
        model.eval()
        running_loss = 0
        running_correct = 0
        x = valid_emd
        y = valid_y
        outputs = model(x)
        loss = criterion(outputs, y)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * x.size(0)
        running_correct += torch.sum(preds==y.data)
        epoch_loss = running_loss / valid_emd.shape[0]
        epoch_acc = running_correct.double() / valid_emd.shape[0]
        print('Validation loss: {:.4f}, accuracy: {:.4f}'.format(epoch_loss, epoch_acc))
        

In [ ]:
tr.criterion = nn.CrossEntropyLoss()

tr.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
).to(device)

tr.optimizer = optim.SGD(tr.fc.parameters(), lr=0.01, momentum=0.9, nesterov=True)

_ = train_embeddings(
    tr.fc, 
    tr.criterion, 
    tr.optimizer, 
    90, 
    tr.train_embeddings, 
    tr.valid_embeddings,
    tr.train_y, 
    tr.valid_y)

## Make Predictions

In [ ]:
orig.model.fc = tr.fc

In [ ]:
# tr.test_images = [
#     input_path + '/walk_or_run_test/test/run/run_78b39d88.png',
#     input_path + '/walk_or_run_test/test/run/run_365fa2e5.png',
#     input_path + '/walk_or_run_test/test/run/run_603ac08a.png',
#     input_path + '/walk_or_run_test/test/walk/walk_9e646bbc.png',
#     input_path + '/walk_or_run_test/test/walk/walk_443d602c.png',
# ]

tr.test_images = [
    OUTPUT_PATH + '/test/healthy/Train_100.jpg',
    OUTPUT_PATH + '/test/multiple_diseases/Train_122.jpg',
    OUTPUT_PATH + '/test/rust/Train_10.jpg',
    OUTPUT_PATH + '/test/rust/Train_102.jpg',
    OUTPUT_PATH + '/test/scab/Train_11.jpg',
]

In [ ]:
from PIL import Image
tr.img_list = [Image.open(img_path).convert("RGB") for img_path in tr.test_images]

In [ ]:
tr.test_batch = torch.stack([
    tr.transforms_valid(img).to(device) for img in tr.img_list])

In [ ]:
tr.logits = orig.model(tr.test_batch)
tr.logits

In [ ]:
from torch.nn import functional as F
tr.proba = F.softmax(tr.logits, dim=1).cpu().data.numpy()
tr.proba

In [ ]:
tr.train_loader.dataset.classes

In [ ]:
tr.fig, tr.axs = plt.subplots(1, len(tr.img_list), figsize=(20, 5))
for i, img in enumerate(tr.img_list):
    ax = tr.axs[i]
    ax.axis('off')
    
    ax.set_title("{:.0f}% {}, {:.0f}% {}\n {:.0f}% {}, {:.0f}% {}"
                 .format(100 * tr.proba[i,0], tr.train_loader.dataset.classes[0],
                         100 * tr.proba[i,1], tr.train_loader.dataset.classes[1],
                         100 * tr.proba[i,2], tr.train_loader.dataset.classes[2],
                         100 * tr.proba[i,3], tr.train_loader.dataset.classes[3],
                  )
    )
    
    ax.imshow(img)

# Save Model for Redis AI

In [ ]:
import ml2rt

This will evaluate and convert model into a special TorchScript format. This format is universal and is longer tied to Python. So, we can load it inside Redis.

In [ ]:
orig.model.eval()
tr.nn_script = torch.jit.trace(orig.model, tr.X_valid[0:10].to(device))

Note: we can use TorchScript model just like any other PyTorch model. Now we can save the model:

In [ ]:
tr.nn_script.eval()
ml2rt.save_torch(tr.nn_script, 'model.pt')

# Load model into Redis AI

In [ ]:
import redisai

In [ ]:
tr.redis = redisai.Client()

In [ ]:
try:
    tr.redis.loadbackend('TORCH', 'redisai_torch/redisai_torch.so')
except:
    pass

In [ ]:
tr.loaded_model = ml2rt.load_model('model.pt')

In [ ]:
tr.redis.modelset(
    "walk_or_run", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

In [ ]:
!ls ../input/walk-or-run/walk_or_run_test/test/walk/

In [ ]:
tr.img = Image \
    .open(input_path + '/walk_or_run_test/test/walk/walk_9d193f21.png') \
    .convert("RGB")
tr.img

Our model expects images in a certain format:
   * 224 x 224
   * Colors re-scaled the mean and std dev:
     * mean=[0.485, 0.456, 0.406]
     * std=[0.229, .224, .225]
   * The image must also be a part of an array, even if it's just one image. We will insert another dimension as the beginning to achieve this.

In [ ]:
tr.img_rescaled = tr.transforms_valid(tr.img)
tr.img_rescaled = np.expand_dims(tr.img_rescaled, axis=0)
tr.img_rescaled.shape

Send the image to Redis AI:

In [ ]:
tr.redis.tensorset('input', tr.img_rescaled)

In [ ]:
tr.redis.modelrun('walk_or_run', 'input', 'pred')

In [ ]:
tr.redis.tensorget('pred')

In [ ]:
torch.softmax(torch.Tensor(tr.redis.tensorget('pred')), dim=1)

Our classes = ['run', 'walk'], so the prediction is **'walk'**.